# MAST TESS Access with Table Access Protocol Service

<br> This notebook is a demo for accessing Transiting Exoplanet Survey Satellite (TESS) data in the Common Archive Observation Model (CAOM) catalog at MAST, using a Virtual Observatory standard Table Access Protocol (TAP) service. 

***
### Table of Contents

1. [TAP Service Introduction](#TAP-Service-Introduction)
2. [Imports](#Imports)
3. [Service Specific Configuration](#Service-Specific-Configuration)
4. [Connecting to the TAP Service](#Connecting-to-the-TAP-Service)
5. [Use Case: Getting light curves from a sector, camera, and chip](Use-Case:-Getting-images-from-a-sector,-camera,-and-chip)
    - [Step 1: Getting the footprint](#Step-1:-Getting-the-footprint)
    - [Step 2: Getting an inventory of TESS lightcurves within the footprint](#Step-2:-Getting-an-inventory-of-TESS-lightcurves-within-the-footprint)
5. [Additional Resources](#Additional-Resources)
6. [About This Notebook](#About-this-Notebook)
***

## TAP Service Introduction

Table Access Protocol (TAP) services allow more direct and flexible access to astronomical data than the simpler types of IVOA standard data services. Queries are built with the SQL-like Astronomical Data Query Language (ADQL), and can include geographic / spatial queries as well as filtering on other characteristics of the data. This also allows the user fine-grained control over the returned columns, unlike the fixed set of coumns retunred from cone, image, and spectral services.

For this example, we'll be using the astroquery TAP/TAP+ client, which was developed by the ESAC Space Data Centre for working with the GAIA catalog, but is interoperable with other valid TAP services, including those at MAST. As an astroquery project, TAP+ documentation is available at ReadTheDocs: http://astroquery.readthedocs.io/en/latest/utils/tap.html

We'll be using TAP+ to call the CAOM Catalog TAP service at MAST and filter the results for TESS-related information. The schema for this catalog is an IVOA standard, and is also described within the service itself.



***
## Imports

In [ ]:
# Use the astroquery TapPlus library as our client to the data service.
from astroquery.utils.tap.core import TapPlus

# For handling ordinary astropy Tables in responses
from astropy.table import Table

# For displaying and manipulating some types of results
import requests
import astropy
import time

# To allow display tweaks for wider response tables
from IPython.core.display import display
from IPython.core.display import HTML

## Service Specific Configuration

Every TAP service has a "Base URL" plus associated endpoints for synchronous and asynchronous queries, as well as status and capability information, and sometimes service-provided sample queries. The endpoints are predefined in the TAP standard, so clients can infer them using the base. We therefore only have to provide astroquery that base.

In [ ]:
TAP_URL = "http://vao.stsci.edu/caomtap/tapservice.aspx"

## Connecting to the TAP Service

The TapPlus library is able to connect to any TAP service, given the "base" URL as noted in metadata registry resources describing the service. The CAOM TAP service at MAST has access to TESS FFI and time series, including file URLs for download.

In [ ]:
TAP_service = TapPlus(url=TAP_URL)

## Use Case: Getting images from a sector, camera, and chip

### Step 1: Getting the footprint

For our purposes, any one footprint related to a sector, camera, and chip combination is good enough. We are not currently accounting for small movements of the spacecraft to form a composite footprint.

In [ ]:
sector = '1'
camera = '1'
chip = '2'
observationIDwildcard = 'tess%-s000' + sector + '-' + camera + '-' + chip + '-%-s'

Here we query for a single ID and footprint to start our search. For filters, we use the TESS mission, target types associated with light curves instead of full frame images, and the sector number. We then use our knowledge of how the IDs are constructed to find one for our sector, camera, and chip combination.

In [ ]:
job = TAP_service.launch_job_async(
            "SELECT top 1 obs_id, s_region " +
            "FROM caomobservation, ivoa.obscore " +
            "WHERE " +
            "observationID = obs_id and " +
            "collection = 'TESS' and trgType = 'FIELD' and " +
            "(sequenceNumber  = " + sector + ") and "
            "observationID like '" + observationIDwildcard + "'"
            )
#print(job.parameters)
footprint_results = job.get_results()
footprint_results

### Step 2: Getting an inventory of TESS lightcurves within the footprint

Here we take the footprint from the above query and find all lightcurves intersecting with this footprint, in ALL sectors. Depending on where this is in the sky, there could be responses only in the original sector, or there could be overlaps with other sectors. There would be more sector overlap near the poles, for instance. By filtering on obs_collection = TESS, we filter based on the TESS mission and exclude High Level Science Products even based on TESS.

The format must be reformatted for our next query. We separate the shape from the list of vertices, which themselves must be comma-separated.

In [ ]:
footprint = footprint_results['s_region'][0].decode('UTF-8')
footprintShape = footprint[0:footprint.find(' ')]
footprintVertices = footprint[footprint.find(' '):].strip().replace(' ', ', ')
print(footprintShape)
print(footprintVertices)


Once the footprint has been isolated and reformatted, we perform another query listing all lightcurves by their target name, sector, RA and Dec, as well as returning the access url for each FITS file and its estimated file size:

In [ ]:
job = TAP_service.launch_job_async(
            "SELECT target_name, sequenceNumber as sector, s_ra, s_dec, access_url, access_estsize " +
            "FROM caomobservation, ivoa.obscore " +
            "WHERE " +
            "observationID = obs_id and " +
            "obs_collection = 'TESS' and " +
            "dataproduct_type = 'timeseries' and " +
            "CONTAINS(POINT('ICRS',s_ra,s_dec)," + footprintShape + "('ICRS', " + footprintVertices + "))" +
            "=1")
TAP_results = job.get_results()
TAP_results          

The returned data is in an astropy table; you can manipulate it to do more ordering or filtering. To download individual files or the whole set, you can use the access_url column, as below:

In [ ]:
fileURL = TAP_results['access_url'][0].decode('UTF-8')
print(fileURL)

***
# Additional Resources


## Table Access Protocol 
* IVOA standard for RESTful web service access to tabular data
* http://www.ivoa.net/documents/TAP/


## Astronomical Query Data Language (2.0)
* IVOA standard for querying astronomical data in tabular format, with geometric search support
* http://www.ivoa.net/documents/latest/ADQL.html


## Common Archive Observation Model (2)
* IVOA standard data model whose relational representation this catalog follows
* http://www.opencadc.org/caom2/


## TapPlus 
* Module created by ESAC Space Data Centre
* http://astroquery.readthedocs.io/en/latest/utils/tap.html

***

## About this Notebook
**Authors:** Scott Fleming & Theresa Dower, STScI Archive Scientists & Software Engineer
**Updated On:** 06/17/2019

***

<img style="float: right;" src="./stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="stsci_pri_combo_mark_horizonal_white_bkgd" width="200px"/> 